In [2]:
USE SCHEMA SilverData;


CREATE TABLE IF NOT EXISTS country (
    country_id BIGINT NOT NULL , --foreign key to trade partner | (TBC)
    country_name STRING, -- (based on ISO2)
    iso          STRING -- COPY from BRONZE(ISO2)

)

USING DELTA;

-- USE bronze_data;

-- SELECT * FROM bronze_data.dim_location;

-- INSERT INTO country (country_id, country_name, iso) 
-- SELECT 
--     LOCATION_KEY AS country_id,

--     CASE 
--         WHEN ISO2 = 'US' THEN 'USA'
--         WHEN ISO2 = 'CA' THEN 'CANADA'
--         ELSE ISO2
--     END AS country_name,

--     ISO2 AS iso
-- FROM bronze_data.dim_location;

-- SELECT * FROM country;


StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
--UPSERTION


CREATE TABLE IF NOT EXISTS  iso_n_country (
    iso STRING,
    country_name STRING
);

MERGE INTO iso_n_country AS tgt
USING (
  SELECT * FROM VALUES
    ('US', 'United States'),
    ('CA', 'Canada'),
    ('IN', 'India'),
    ('AE', 'United Arab Emirates'),
    ('GB', 'United Kingdom'),
    ('CN', 'China'),
    ('JP', 'Japan')
  AS src(iso, country_name)
) AS src
ON tgt.iso = src.iso
WHEN MATCHED THEN
  UPDATE SET tgt.country_name = src.country_name
WHEN NOT MATCHED THEN
  INSERT (iso, country_name)
  VALUES (src.iso, src.country_name);

-- Note as Delta tables doesn't support Primary keys  as well as IDENTITY generation, we have no option left except hash 
MERGE INTO country AS tgt
USING (
    SELECT DISTINCT
        ABS(HASH(dl.ISO2)) AS country_id,
        COALESCE(c.country_name, dl.ISO2) AS country_name,
        dl.ISO2 AS iso
    FROM bronze_data.dim_location AS dl
    LEFT JOIN iso_n_country AS c
        ON c.iso = dl.ISO2
) AS src
ON tgt.iso = src.iso
WHEN NOT MATCHED THEN
    INSERT (country_id, country_name, iso)
    VALUES (src.country_id, src.country_name, src.iso);



StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

In [4]:
USE SCHEMA SilverData;


CREATE TABLE IF NOT EXISTS reporter (
    reporter_id  BIGINT NOT NULL , -- (Location Key) | foreign key of country
    reporter_name STRING, --(Just Onario for rn)
    country_id BIGINT --foreign key to Trades | (TBC)
)

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
--UPSERTION to reporter

--logic {reporter_id == dim_loction.location_key,reporter_name== dim_location.Name,
-- country_id= hash(ISO2) or use countryid where country.iso==dim_location.ISO2}
----MERGE-----

-- TRUNCATE TABLE reporter;

MERGE INTO reporter r
USING(
        SELECT  
        dl.LOCATION_KEY as reporter_id ,
        dl.Name as  reporter_name,
        c.country_id as country_id
FROM country AS c
JOIN bronze_data.dim_location AS dl ON c.iso=dl.ISO2
)src 
        ON r.reporter_id=src.reporter_id
        AND r.reporter_name=src.reporter_name
        AND r.country_id=src.country_id
WHEN NOT MATCHED THEN 
INSERT  (reporter_id, reporter_name,country_id) 
VALUES(src.reporter_id,src.reporter_name, country_id);



---INSERT-----

-- INSERT INTO reporter (reporter_id, reporter_name,country_id) 
-- SELECT  
--         dl.LOCATION_KEY,
--         dl.Name,
--         c.country_id
-- FROM country AS c
-- JOIN bronze_data.dim_location AS dl ON c.iso=dl.ISO2;

-- SELECT * FROM reporter;

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [6]:
CREATE TABLE IF NOT EXISTS trade_partner (
    trade_state_id BIGINT NOT NULL , --foreign key to trades  | ()
    state_name STRING,  -- All  us States (Names with ISO2=US)
    country_id BIGINT  --Foreign key of reporter | 
)

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
--UPSERTION TO trade_partner



---MERGE---

-- TRUNCATE TABLE trade_partner;
MERGE INTO trade_partner tdp 
USING(
    SELECT  dl.LOCATION_KEY AS  trade_state_id,
    dl.NAME AS state_name,
    r.country_id AS country_id
    FROM reporter AS r
    JOIN bronze_data.dim_location AS dl ON r.reporter_name = dl.NAME
) src
ON tdp.trade_state_id=src.trade_state_id
AND tdp.state_name=src.state_name
AND tdp.country_id=src.country_id

WHEN NOT MATCHED THEN 
INSERT (trade_state_id,state_name,country_id) 
VALUES (
    src.trade_state_id,
    src.state_name,
    src.country_id
);



-- SELECT * FROM trade_partner;



StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 12, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [8]:
--Upsertion to hs_section



---MERGE----

-- TRUNCATE TABLE hs_section;

MERGE INTO hs_section hss
USING (
    SELECT 
    dhp.HS_CODE AS section_code ,
    dhp.DESCRIPTION AS description
FROM bronze_data.dim_hs_product as dhp
) src 
ON hss.section_code=src.section_code
AND hss.description = src.description
WHEN NOT MATCHED THEN 
INSERT (section_id,section_code,description)
VALUES(
    ABS(HASH(src.section_code,src.section_code)),
    src.section_code,src.description
);



StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 13, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [9]:

CREATE TABLE IF NOT EXISTS trade_flow_type (
 trade_flow_type_id BIGINT NOT NULL, --fk of TradeFlowType 
 trade_flow_type_name STRING 
)

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 14, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
--Upsertion to trade_flow_type

---MERGE----
MERGE INTO trade_flow_type tft 
USING (
    SELECT df.FLOW_CODE AS trade_flow_type_id ,
           df.FLOW_DESC AS trade_flow_type_name
           FROM bronze_data.dim_flow AS df
) 
src 
ON tft.trade_flow_type_id = src.trade_flow_type_id
AND tft.trade_flow_type_name = src.trade_flow_type_name

WHEN NOT MATCHED THEN 
    INSERT(trade_flow_type_id,trade_flow_type_name) 
    VALUES (src.trade_flow_type_id,src.trade_flow_type_name);




StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 15, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [11]:


CREATE TABLE IF NOT EXISTS trades (
    trades_id BIGINT NOT NULL, --foreign key to trades_details
    date DATE,
    trade_state_id BIGINT, --fk of trade_partner | partner_key of bronze_data.factrade
    reporter_id BIGINT, --fk of Reporter | reporter_key of bronze_data.factrade
    trade_flow_type_id BIGINT --fk of TradeFlowType
)


StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 16, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
--UPSERTION FOR trades 

-- TRUNCATE TABLE trades;
-----MERGE -----
-- CHECKS IF NEW DISTINCT RECORD APPENDED IN BRONZE IF YES THEN INSERTS THEM
MERGE INTO trades t

USING (
    SELECT DISTINCT
        TO_DATE(TRADE_DATE, 'yyyy-MM-dd') AS date,
        PARTNER_KEY AS trade_state_id,
        REPORTER_KEY AS reporter_id,
        FLOW_CODE AS trade_flow_type_id
    FROM bronze_data.fact_trade
) b

ON t.date=b.date
AND t.trade_state_id=b.trade_state_id
AND t.reporter_id=b.reporter_id
AND t.trade_flow_type_id = b.trade_flow_type_id

WHEN NOT MATCHED THEN 
    INSERT(trades_id,date,trade_state_id,reporter_id,trade_flow_type_id)
    VALUES(
        ABS(HASH(b.date,b.trade_state_id,b.reporter_id,b.trade_flow_type_id)),
        b.date, b.trade_state_id, b.reporter_id, b.trade_flow_type_id
    );


-- SELECT COUNT(*) FROM trades ;

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 17, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [13]:
CREATE TABLE IF NOT EXISTS trade_details (
    trade_details_id BIGINT NOT NULL , 
    trades_id BIGINT,--foreign key of trades
    hs_sections_id BIGINT, --fk of HS_SECTION
    dollar_value INT, 
    iso STRING --fk of Reporter
)

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, -1, Finished, Available, Finished)

In [15]:

-------MERGE------


MERGE INTO trade_details td
USING (
    SELECT 
        ABS(HASH(t.trades_id, hs.section_id, dl.ISO2)) AS trade_details_id,
        t.trades_id,
        hs.section_id  AS hs_sections_id,
        ft.VALUE      AS dollar_value,
        dl.ISO2       AS iso
    FROM trades AS t
    JOIN bronze_data.fact_trade AS ft
        ON  TO_DATE(ft.TRADE_DATE, 'yyyy-MM-dd') = t.date
        AND ft.REPORTER_KEY = t.reporter_id
        AND ft.PARTNER_KEY = t.trade_state_id 
        AND ft.FLOW_CODE  = t.trade_flow_type_id
    JOIN hs_section AS hs 
        ON hs.section_code = CAST(ft.HS_CODE AS STRING)
    JOIN bronze_data.dim_location AS dl
        ON dl.LOCATION_KEY = ft.PARTNER_KEY
) src
ON  td.trades_id = src.trades_id
AND td.hs_sections_id = src.hs_sections_id
AND td.iso = src.iso
AND td.dollar_value = src.dollar_value
WHEN NOT MATCHED THEN
  INSERT (trade_details_id, trades_id, hs_sections_id, dollar_value, iso)
  VALUES (src.trade_details_id, src.trades_id, src.hs_sections_id, src.dollar_value, src.iso);




StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 19, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [16]:
select count(*) from trades;

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 20, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [17]:
select count(*) from trade_details;


StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 21, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [18]:
SELECT * from bronze_data.dim_location; 

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 22, Finished, Available, Finished)

<Spark SQL result set with 52 rows and 3 fields>

StatementMeta(, c313f1b4-91cc-40c1-8c3b-950d0f33c06f, 23, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 3 fields>